In [ ]:
import json
from collections import Counter
import regex as re
import os
import pandas as pd
from tokenizers import Tokenizer
import pyphen
import matplotlib.pyplot as plt
import pandas as pd

# Analyze Masked words

In [ ]:
file_to_train='strict_model'
file_to_save='7_full_data__sum_freq_gramma__max_freq'
path='./SlovakBabyLM/Curricullum_learning/'

In [ ]:
with open(f'{path}{file_to_train}_results/{file_to_save}/masked_words.txt', "r", encoding="utf8") as f:
    data = f.readlines()

In [20]:
word_counter = Counter()


for word in data: 
    word=word.replace("\n","")
    if word:
        word_counter[word] += 1
top_10 = word_counter.most_common(10)

In [21]:
len(word_counter)

17266

In [23]:
df_top_10_max = pd.DataFrame(top_10, columns=['Word', 'Frequency'])
df_top_10_max

,Word,Frequency
0,sa,258273
1,a,207047
2,to,159399
3,je,131987
4,na,114932
5,v,104538
6,si,96317
7,som,68440
8,že,48899
9,s,38767


In [22]:
df_top_10_min = pd.DataFrame(top_10, columns=['Word', 'Frequency'])
df_top_10_min

,Word,Frequency
0,ľúto,142
1,istý,139
2,rozhodovaní,130
3,myslel,128
4,myslela,121
5,nepovedal,121
6,vedel,120
7,vedela,119
8,nepovedala,118
9,istá,114


# how many masks are in text 

In [ ]:
file_to_train='strict_model'
file_to_save='6_full_data__sum_freq_gramma__min_freq'

with open(f'{path}{file_to_train}_results/{file_to_save}/final_masked_text.txt', "r", encoding="utf8") as f:
    data = f.read()


In [4]:
data.count('[MASK]')

2579958

Open and analyze results

In [ ]:
import json
with open(f'{path}results_evaluation/strict_model/output_sentiment.json', 'r') as file:
    data = json.load(file)

In [6]:
from collections import defaultdict
import statistics

In [7]:
grouped = defaultdict(list)

for item in data:
    for full_key, metrics in item.items():
        grouped[full_key].append(metrics)

AttributeError: 'str' object has no attribute 'items'

In [5]:
rows = []
for model, entries in grouped.items():
    exact_matches = [e["exact_match"] for e in entries]
    f1 = [e["f1"] for e in entries]
    
    row = {
        "model": model,
        "exact_match_avg": statistics.mean(exact_matches),
        "exact_match_std": statistics.stdev(exact_matches) if len(exact_matches) > 1 else 0.0,
        "f1_avg": statistics.mean(f1),
        "f1_std": statistics.stdev(f1) if len(f1) > 1 else 0.0,
    }
    rows.append(row)

df = pd.DataFrame(rows)

In [6]:
df

,model,exact_match_avg,exact_match_std,f1_avg,f1_std
0,randomly_selected_subs,1.613623,0.460422,7.055418,0.902425
1,randomly_selected_wiki_full,1.072516,0.175445,6.126928,0.698651


In [ ]:
with open(f'{path}results_evaluation/strict_model_ordered_data/output_sentiment.json', 'r') as file:
    data_2 = json.load(file)

In [17]:
grouped_data_2 = {}

for full_key, records in data_2.items():
    grouped_data_2.setdefault(full_key, []).extend(records)


In [18]:
rows = []
for main_key, records in grouped_data_2.items():
    row = {
        "main_key": main_key,
        "eval_loss_avg": statistics.mean([r["eval_loss"] for r in records]),
        "eval_loss_std": statistics.stdev([r["eval_loss"] for r in records]) if len(records) > 1 else 0.0,
        "eval_accuracy_avg": statistics.mean([r["eval_accuracy"] for r in records]),
        "eval_accuracy_std": statistics.stdev([r["eval_accuracy"] for r in records]) if len(records) > 1 else 0.0,
        "eval_precision_avg": statistics.mean([r["eval_precision"] for r in records]),
        "eval_precision_std": statistics.stdev([r["eval_precision"] for r in records]) if len(records) > 1 else 0.0,
        "eval_recall_avg": statistics.mean([r["eval_recall"] for r in records]),
        "eval_recall_std": statistics.stdev([r["eval_recall"] for r in records]) if len(records) > 1 else 0.0,
        "eval_f1_avg": statistics.mean([r["eval_f1"] for r in records]),
        "eval_f1_std": statistics.stdev([r["eval_f1"] for r in records]) if len(records) > 1 else 0.0,
    }
    rows.append(row)

df = pd.DataFrame(rows)

In [15]:
df

,main_key,eval_loss_avg,eval_loss_std,eval_accuracy_avg,eval_accuracy_std,eval_precision_avg,eval_precision_std,eval_recall_avg,eval_recall_std,eval_f1_avg,eval_f1_std
0,1_full_data__without_ord__without_mask,0.321154,0.100983,0.903178,0.026126,0.854549,0.081278,0.903178,0.026126,0.877263,0.055262


In [19]:
df

,main_key,eval_loss_avg,eval_loss_std,eval_accuracy_avg,eval_accuracy_std,eval_precision_avg,eval_precision_std,eval_recall_avg,eval_recall_std,eval_f1_avg,eval_f1_std
0,2_files_data__sum_freq_gramma__without_mask,0.334457,0.092024,0.897206,0.021360,0.846846,0.074387,0.897206,0.021360,0.870180,0.049077
1,3_full_data__sum_freq_gramma__without_mask,0.335180,0.100665,0.895713,0.024126,0.834823,0.078342,0.895713,0.024126,0.863243,0.052623
2,1_full_data__without_ord__without_mask,0.326032,0.100120,0.900512,0.024250,0.850499,0.077584,0.900512,0.024250,0.873586,0.051998


In [ ]:
with open(f'{path}results_evaluation/strict_model/output_evaluation.json', 'r') as file:
    data_2 = json.load(file)

In [8]:
rows = {}
for config, entries in data_2.items():
    row = {}
    for entry in entries:
        row[entry['dataset']] = entry['eval_loss']
    rows[config] = row

df = pd.DataFrame.from_dict(rows, orient='index')


In [9]:
df

,wiki_full,subs,strict-model
7_full_data__sum_freq_gramma__min_freq,8.823563,6.972857,7.723399
6_full_data__sum_freq_gramma__max_freq,8.767971,7.157136,7.823065
5_full_data__freq_group__without_mask,8.553845,6.703813,7.425178
2_files_data__sum_freq_gramma__without_mask,8.561524,6.707302,7.435490
3_full_data__sum_freq_gramma__without_mask,8.542048,6.705210,7.421519
4_full_data__gramma_group__without_mask,8.536770,6.702776,7.416385
1_full_data__without_ord__without_mask,8.493325,6.672991,7.404138
